In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
!wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv

--2025-10-12 05:21:48--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80876 (79K) [text/plain]
Saving to: ‘course_lead_scoring.csv.2’

course_lead_scoring 100%[===================>]  78.98K  --.-KB/s    in 0.007s  

2025-10-12 05:21:48 (10.6 MB/s) - ‘course_lead_scoring.csv.2’ saved [80876/80876]



In [2]:
df = pd.read_csv('course_lead_scoring.csv')

In [3]:
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [4]:
categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

In [5]:
categorical_columns

['lead_source', 'industry', 'employment_status', 'location']

In [6]:
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

Data preparation
Check if the missing values are presented in the features.
If there are missing values:
For caterogiral features, replace them with 'NA'
For numerical features, replace with with 0.0

In [7]:
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].fillna('NA')
    else:
        df[col] = df[col].fillna(0.0)

In [8]:
df.isnull().sum()

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

Question 1
What is the most frequent observation (mode) for the column industry?

NA
technology
healthcare
retail

In [9]:
mode_value = df['industry'].mode()[0]
mode_value

'retail'

In [10]:
frequencies = df['industry'].value_counts()
frequencies

industry
retail           203
finance          200
other            198
healthcare       187
education        187
technology       179
manufacturing    174
NA               134
Name: count, dtype: int64

Question 2
Create the correlation matrix for the numerical features of your dataset. In a correlation matrix, you compute the correlation coefficient between every pair of features.

What are the two features that have the biggest correlation?

interaction_count and lead_score
number_of_courses_viewed and lead_score
number_of_courses_viewed and interaction_count
annual_income and interaction_count
Only consider the pairs above when answering this question.

Split the data
Split your data in train/val/test sets with 60%/20%/20% distribution.
Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.
Make sure that the target value y is not in your dataframe.

In [89]:
from sklearn.model_selection import train_test_split

In [90]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

In [91]:
len(df_train), len(df_val), len(df_test)

(876, 293, 293)

In [92]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [93]:
y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

del df_train['converted']
del df_val['converted']
del df_test['converted']

In [11]:
X_numeric = df.select_dtypes(include='number')
X_numeric

,number_of_courses_viewed,annual_income,interaction_count,lead_score,converted
0,1,79450.0,4,0.94,1
1,1,46992.0,1,0.80,0
2,5,78796.0,3,0.69,1
3,2,83843.0,1,0.87,0
4,3,85012.0,3,0.62,1
...,...,...,...,...,...
1457,1,0.0,4,0.53,1
1458,3,65259.0,2,0.24,1
1459,1,45688.0,3,0.02,1
1460,5,71016.0,0,0.25,1


In [12]:
corr_matrix = X_numeric.corr()
corr_matrix

,number_of_courses_viewed,annual_income,interaction_count,lead_score,converted
number_of_courses_viewed,1.000000,0.009770,-0.023565,-0.004879,0.435914
annual_income,0.009770,1.000000,0.027036,0.015610,0.053131
interaction_count,-0.023565,0.027036,1.000000,0.009888,0.374573
lead_score,-0.004879,0.015610,0.009888,1.000000,0.193673
converted,0.435914,0.053131,0.374573,0.193673,1.000000


In [13]:
pairs = [
    ('interaction_count', 'lead_score'),
    ('number_of_courses_viewed', 'lead_score'),
    ('number_of_courses_viewed', 'interaction_count'),
    ('annual_income', 'interaction_count')
]

max_corr = 0
max_pair = None

for f1, f2 in pairs:
    corr = corr_matrix.loc[f1, f2]
    if abs(corr) > abs(max_corr):
        max_corr = corr
        max_pair = (f1, f2)

max_pair

('annual_income', 'interaction_count')

Question 3
Calculate the mutual information score between y and other categorical variables in the dataset. Use the training set only.
Round the scores to 2 decimals using round(score, 2).
Which of these variables has the biggest mutual information score?

industry
location
lead_source
employment_status

In [31]:
from sklearn.metrics import mutual_info_score

In [32]:
df_full_train = df_full_train.reset_index(drop=True)

In [43]:
def mutual_info_converted_score(series):
    return mutual_info_score(series, df_full_train.converted)

In [44]:
categorical = [
    'lead_source',
    'industry',
    'employment_status',
    'location'
]

In [46]:
numerical = ['number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score']

In [45]:
mi = df_full_train[categorical].apply(mutual_info_converted_score)
mi = round(mi, 2)
mi.sort_values(ascending=False)

lead_source          0.03
industry             0.01
employment_status    0.01
location             0.00
dtype: float64

Question 4
Now let's train a logistic regression.
Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.
Fit the model on the training dataset.
To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
Calculate the accuracy on the validation dataset and round it to 2 decimal digits.
What accuracy did you get?

0.64
0.74
0.84
0.94

In [96]:
from sklearn.feature_extraction import DictVectorizer

In [97]:
dv = DictVectorizer(sparse=False)

train_dict = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [98]:

from sklearn.linear_model import LogisticRegression

In [99]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'liblinear'
,max_iter,1000
,multi_class,'deprecated'


In [100]:
y_pred = model.predict_proba(X_val)[:, 1]

In [101]:
converted = (y_pred >= 0.5)

In [103]:
accuracy = (y_val == converted).mean()
accuracy = round(accuracy, 2)
accuracy

np.float64(0.7)

Question 5

Let's find the least useful feature using the feature elimination technique.
Train a model using the same features and parameters as in Q4 (without rounding).
Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
For each feature, calculate the difference between the original accuracy and the accuracy without the feature.
Which of following feature has the smallest difference?

'industry'
'employment_status'
'lead_score'
Note: The difference doesn't have to be positive.

In [104]:
dicts_full_train = df_full_train[categorical + numerical].to_dict(orient='records')

In [105]:
dv = DictVectorizer(sparse=False)
X_full_train = dv.fit_transform(dicts_full_train)

In [106]:
y_full_train = df_full_train.converted.values

In [107]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_full_train, y_full_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'liblinear'
,max_iter,1000
,multi_class,'deprecated'


In [108]:
dicts_test = df_test[categorical + numerical].to_dict(orient='records')

In [109]:
X_test = dv.transform(dicts_test)

In [110]:
y_pred = model.predict_proba(X_test)[:, 1]

In [111]:
converted = (y_pred >= 0.5)

In [112]:
accuracy_original = (y_test == converted).mean()
accuracy_original

np.float64(0.7372013651877133)

In [113]:
features = ['lead_source',
            'industry',
            'employment_status',
            'location',
            'number_of_courses_viewed', 
            'annual_income', 
            'interaction_count', 
            'lead_score']

In [116]:
diffs = {}

for f in features:
    subset = [col for col in features if col != f]
    
    dicts_full_train = df_full_train[subset].to_dict(orient='records')
    dv = DictVectorizer(sparse=False)
    X_full_train = dv.fit_transform(dicts_full_train)
    
    model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    model.fit(X_full_train, y_full_train)

    dicts_test = df_test[subset].to_dict(orient='records')
    X_test = dv.transform(dicts_test)
    y_pred = model.predict_proba(X_test)[:, 1]
    converted = (y_pred >= 0.5)

    accuracy = (y_test == converted).mean()
    diffs[f] = accuracy_original - accuracy
    print(f'{f}: {accuracy:.4f} (diff {accuracy_original - accuracy:.4f})')

lead_source: 0.7270 (diff 0.0102)
industry: 0.7372 (diff 0.0000)
employment_status: 0.7270 (diff 0.0102)
location: 0.7372 (diff 0.0000)
number_of_courses_viewed: 0.6689 (diff 0.0683)
annual_income: 0.8191 (diff -0.0819)
interaction_count: 0.7099 (diff 0.0273)
lead_score: 0.7406 (diff -0.0034)


Question 6
Now let's train a regularized logistic regression.
Let's try the following values of the parameter C: [0.01, 0.1, 1, 10, 100].
Train models using all the features as in Q4.
Calculate the accuracy on the validation dataset and round it to 3 decimal digits.
Which of these C leads to the best accuracy on the validation set?

0.01
0.1
1
10
100
Note: If there are multiple options, select the smallest C.

In [122]:
dicts_full_train = df_full_train[categorical + numerical].to_dict(orient='records')

In [123]:
dv = DictVectorizer(sparse=False)
X_full_train = dv.fit_transform(dicts_full_train)

In [124]:
y_full_train = df_full_train.converted.values

In [125]:
C_values = [0.01, 0.1, 1, 10, 100]

In [126]:
for C in C_values:
    model = LogisticRegression(solver='liblinear', C=C, max_iter=1000, random_state=42)
    model.fit(X_full_train, y_full_train)
    dicts_test = df_test[categorical + numerical].to_dict(orient='records')
    X_test = dv.transform(dicts_test)
    y_pred = model.predict_proba(X_test)[:, 1]
    converted = (y_pred >= 0.5)
    accuracy = (y_test == converted).mean()
    accuracy
    print(f'C={C}: accuracy={accuracy:.3f}')

C=0.01: accuracy=0.734
C=0.1: accuracy=0.737
C=1: accuracy=0.737
C=10: accuracy=0.737
C=100: accuracy=0.737
